Coloring regions to blur

Drawing a boundary to blur

In [22]:
import gradio as gr
import torch
import torch.nn as nn
import numpy as np
from PIL import Image
import cv2
import os
from scipy.ndimage import binary_fill_holes


# ============================================================
#                    FAST BLURNET (NO TRIPLE PASSES)
# ============================================================
class BlurNet(nn.Module):
    """CNN-based blurring network with single-pass layers."""
    def __init__(self, blur_intensity="medium"):
        super(BlurNet, self).__init__()

        intensity_config = {
            "mild":   {"layers": 3, "sigma": 3.0, "kernel": 7},
            "medium": {"layers": 5, "sigma": 5.0, "kernel": 9},
            "strong": {"layers": 7, "sigma": 7.0, "kernel": 11}
        }

        config = intensity_config[blur_intensity]
        layers = config["layers"]
        sigma = config["sigma"]
        kernel = config["kernel"]

        self.blur_layers = nn.ModuleList()

        print(f"\n[BlurNet INIT] Mode={blur_intensity}")
        for i in range(layers):
            padding = kernel // 2
            conv = nn.Conv2d(
                3, 3,
                kernel_size=kernel,
                stride=1,
                padding=padding,
                groups=3,
                bias=False
            )

            dyn_sigma = sigma * (1 + 0.20 * i)
            print(f"  Layer {i+1}: kernel={kernel}, sigma={dyn_sigma:.2f}")

            self._init_kernel(conv, dyn_sigma, kernel)
            self.blur_layers.append(conv)

    def _init_kernel(self, conv, sigma, kernel_size):
        ax = np.arange(-kernel_size//2 + 1., kernel_size//2 + 1.)
        xx, yy = np.meshgrid(ax, ax)
        g = np.exp(-(xx**2 + yy**2) / (2*sigma*sigma))
        g = g / g.sum()

        kernel_tensor = torch.from_numpy(g).float()
        for i in range(3):
            conv.weight.data[i, 0, :, :] = kernel_tensor

    def forward(self, x):
        print(f"[BlurNet] Running {len(self.blur_layers)} layers...")
        for idx, layer in enumerate(self.blur_layers):
            print(f"  Executing layer {idx+1}/{len(self.blur_layers)}")
            x = layer(x)
        return x


# ============================================================
#               INITIALIZE MODELS
# ============================================================
print("Initializing BlurNet models...\n")
models = {
    "mild": BlurNet("mild").eval(),
    "medium": BlurNet("medium").eval(),
    "strong": BlurNet("strong").eval()
}
print("\nAll models initialized.\n")

os.makedirs("blur_outputs", exist_ok=True)
counter = 0


# ============================================================
#                  IMAGE HELPERS
# ============================================================
def resize_if_large(image, max_dim=1200):
    width, height = image.size
    if max(width, height) > max_dim:
        if width > height:
            new_width = max_dim
            new_height = int(height * (max_dim / width))
        else:
            new_height = max_dim
            new_width = int(width * (max_dim / height))

        print(f"Resizing image: ({width}, {height}) → ({new_width}, {new_height})")
        image = image.resize((new_width, new_height), Image.LANCZOS)
    return image


def fill_enclosed_region(mask):
    print("Filling enclosed region...")
    orig = int(mask.sum())
    filled = binary_fill_holes(mask).astype(bool)
    new = int(filled.sum())
    print(f"Interior fill: {orig} → {new} pixels")
    return filled


# ============================================================
#           UNIFIED BLUR PIPELINE (ALL LEVELS)
# ============================================================
def apply_intensity_blur(blurred, mode):
    if mode == "mild":
        gk, bk = 25, 15
    elif mode == "medium":
        gk, bk = 45, 25
    else:
        gk, bk = 65, 45

    if gk % 2 == 0: gk += 1
    if bk % 2 == 0: bk += 1

    print(f"Applying {mode} blur...")
    print(f"  Gaussian kernel={gk}")
    print(f"  Box kernel={bk}")

    blurred = cv2.GaussianBlur(blurred, (gk, gk), 0)
    blurred = cv2.blur(blurred, (bk, bk))

    return blurred


# ============================================================
#           MAIN PROCESSING FUNCTION (DUAL LOGGING)
# ============================================================
def apply_selective_blur(image_dict, blur_intensity, feather_amount, fill_interior):

    global counter

    # Logs for UI
    ui_logs = []

    def log(msg):
        print(msg)               # visible in terminal
        ui_logs.append(msg)      # returned to UI

    log("\n============================================================")
    log("PROCESSING STARTED")
    log("============================================================")

    if image_dict is None:
        return None, "No image provided."

    original_image = image_dict.get("background") or image_dict.get("image")
    original_image = resize_if_large(original_image)
    original = np.array(original_image)
    log(f"Image shape: {original.shape}")

    # Convert alpha → RGB
    if original.shape[2] == 4:
        log("Converting RGBA to RGB")
        original = original[:, :, :3]

    # Mask extraction
    mask_layer = image_dict["layers"][0] if ("layers" in image_dict and image_dict["layers"]) else None

    if mask_layer is None:
        return Image.fromarray(original), "No mask drawn."

    mask_layer = resize_if_large(mask_layer)
    mask_np = np.array(mask_layer)

    if mask_np.shape[2] == 4:
        mask = mask_np[:, :, 3] > 0
    else:
        mask = np.any(mask_np > 0, axis=2)

    drawn_pixels = int(mask.sum())
    log(f"Mask drawn pixels: {drawn_pixels}")

    if drawn_pixels == 0:
        return Image.fromarray(original), "Empty mask."

    if fill_interior:
        mask = fill_enclosed_region(mask)

    # Resize mask to match image
    if mask.shape != original.shape[:2]:
        log("Resizing mask to match image")
        mask = cv2.resize(mask.astype(np.uint8),
                          (original.shape[1], original.shape[0]),
                          interpolation=cv2.INTER_LINEAR).astype(bool)

    feathered_mask = mask.astype(float)

    # Feathering
    if feather_amount > 0:
        kernel = feather_amount * 2 + 1
        log(f"Applying feathering: kernel={kernel}")
        feathered_mask = cv2.GaussianBlur(feathered_mask, (kernel, kernel), 0)
        feathered_mask = np.clip(feathered_mask, 0, 1)

    # BlurNet
    log(f"Running BlurNet for mode '{blur_intensity}'")
    tensor_image = torch.from_numpy(original).float().permute(2, 0, 1) / 255.
    tensor_image = tensor_image.unsqueeze(0)

    model = models[blur_intensity]
    with torch.no_grad():
        blurred_tensor = model(tensor_image)

    blurred = blurred_tensor.squeeze().permute(1, 2, 0).numpy()
    blurred = (blurred * 255).clip(0, 255).astype(np.uint8)

    # Unified blur pipeline
    blurred = apply_intensity_blur(blurred, blur_intensity)

    # Blend
    log("Blending final result...")
    mask3 = np.stack([feathered_mask] * 3, axis=2)
    result = (original * (1 - mask3) + blurred * mask3).astype(np.uint8)

    counter += 1
    output_path = f"blur_outputs/output_{blur_intensity}_{counter}.png"
    Image.fromarray(result).save(output_path)

    log(f"Saved output to: {output_path}")
    log("PROCESSING COMPLETE")
    log("============================================================")

    ui_log_text = "\n".join(ui_logs)
    return Image.fromarray(result), ui_log_text


# ============================================================
#                       GRADIO UI
# ============================================================
css = """
button.primary {
    background-color: #ff8c00 !important;
    color: white !important;
}
"""

with gr.Blocks(title="Selective Blur Tool", css=css) as demo:

    gr.Markdown("### Selective Region Blur Tool\nDraw a closed region to blur it.")

    with gr.Row():
        with gr.Column():
            editor = gr.ImageEditor(
                label="Draw around the region you want to blur",
                type="pil",
                brush=gr.Brush(colors=["#FFFFFF"], default_size=15),
                eraser=gr.Eraser(default_size=15),
                height=450
            )

            blur_intensity = gr.Radio(
                ["mild", "medium", "strong"],
                value="medium",
                label="Blur Intensity"
            )

            feather_amount = gr.Slider(
                0, 50, value=20,
                label="Edge Feathering"
            )

            fill_interior = gr.Checkbox(
                value=True,
                label="Fill Enclosed Region"
            )

            btn = gr.Button("Apply Blur", variant="primary")

        with gr.Column():
            output_image = gr.Image(
                label="Blurred Output",
                type="pil",
                height=450,
                show_download_button=True
            )

            output_log = gr.Textbox(
                label="Processing Log",
                lines=12,
                interactive=False
            )

    btn.click(
        fn=apply_selective_blur,
        inputs=[editor, blur_intensity, feather_amount, fill_interior],
        outputs=[output_image, output_log]
    )


demo.launch(share=True)


Initializing BlurNet models...


[BlurNet INIT] Mode=mild
  Layer 1: kernel=7, sigma=3.00
  Layer 2: kernel=7, sigma=3.60
  Layer 3: kernel=7, sigma=4.20

[BlurNet INIT] Mode=medium
  Layer 1: kernel=9, sigma=5.00
  Layer 2: kernel=9, sigma=6.00
  Layer 3: kernel=9, sigma=7.00
  Layer 4: kernel=9, sigma=8.00
  Layer 5: kernel=9, sigma=9.00

[BlurNet INIT] Mode=strong
  Layer 1: kernel=11, sigma=7.00
  Layer 2: kernel=11, sigma=8.40
  Layer 3: kernel=11, sigma=9.80
  Layer 4: kernel=11, sigma=11.20
  Layer 5: kernel=11, sigma=12.60
  Layer 6: kernel=11, sigma=14.00
  Layer 7: kernel=11, sigma=15.40

All models initialized.

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4e363380e6e7e4d9cd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
